In [13]:
import requests
import pandas as pd
import re
import io
import json
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from fiona.crs import from_epsg
import IPython.display as display
from joblib import Parallel, delayed
from multiprocessing import Process, Pool
from timeit import default_timer as timer
%matplotlib inline

In [2]:
def read_file(year):
    fileyear = int(year) + 89
    path = 'Data/estate/' + str(fileyear) + '.csv'
    df = pd.read_csv(path, encoding = 'BIG5').rename(
    columns={'AA16': 'market_value', 
             'AA45': 'city', 
             'AA46': 'district',
             'AA48': 'road_seg',
             'AA49': 'seg_num'})
    df.seg_num = df.seg_num.astype('str')
    df = df[['city', 'district', 'road_seg', 'seg_num', 'market_value']]
    return df

In [3]:
def jiduan(s):
    if s[-1] == '0':
        r = s.strip("0")
    else:
        r = re.sub('0' + '{2,}', '-', s)
    return r


In [4]:
def output_shp(data, filename):
    j = json.loads(data)
    df = pd.io.json.json_normalize(j)
    df.coordinates = df.coordinates.apply(
        lambda x: list(map(tuple, x[0][0])))
    df['coordinates'] = df['coordinates'].apply(
        lambda x: Polygon(x))
    gdf = gpd.GeoDataFrame(df, crs=from_epsg(4326), 
                           geometry='coordinates')
    gdf.to_file('Data/shp/' + filename, encoding='BIG5')

In [104]:
dfs = read_file('17')

In [105]:
# dfs = dfs.sample(n=1000, random_state=99)

In [107]:
len(dfs)

414172

In [109]:
batch = 80
num_batch = int(len(dfs) // batch + 1)


api = 'http://twland.ronny.tw/index/search?'
search = 'lands[]=臺北市,'
outter = []
#num_batch = len(dfs) // batch + 1
start = int(0)
end = int(80)
st = int(0)
e = int(80)
null = None

def get_shp(n): 
    global start, end, st, e, outter
    try:
        entry = ''
        if n % 10 == 0:
            print('retrieving: ', (n + 1) * batch, ' / ', len(dfs))

        for i in range(start, end):
            if end > len(dfs):
                print('end {}, len {}'.format(end, len(dfs)))
                print('1 truncate last batch')
                break            

            r = dfs.iloc[i,].road_seg.strip()
            s = dfs.iloc[i,].seg_num
            s = jiduan(s)
            entry += search + r + ',' + s + '&'
#         print('start %s' %start, 'end %s' %end)
        start = int(end) #batch tracker
        end = int(start + batch)

        data = eval(requests.get(api + entry).content)

        #parse and mutate
        length = int(len(data['features']))
    #         print('length %s' %length)
        l = st + length
        for idx, j in enumerate(range(st, l)):
            try:
        #             print('j %s'%j)
                if l > len(dfs):
                    print('end {}, len {}'.format(l, len(dfs)))
                    print('2 truncate last batch')
                    break              
#                     data['features'][idx]['properties']['market_value'] = int(dfs.iloc[j,].market_value)
                inner = data['features'][idx]['geometry']
                inner.update(data['features'][idx]['properties'])
                #print('inner %s' %inner)
                outter.append(inner)
            except AttributeError:
                print('AttributeError')
                continue
        st = int(e)
        e = int(st + batch)
#         print('outter %s' % outter)
    except SyntaxError:
        print('SyntaxError')
        pass
        
    print('done')
    return json.dumps(outter)
    
    

In [51]:
test = get_shp(100)

retrieving:  8080  /  1000
1645.482876165 240 <class 'float'> <class 'int'>


TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
batch = 80
num_batch = int(len(dfs) // batch + 1)


api = 'http://twland.ronny.tw/index/search?'
search = 'lands[]=臺北市,'
outter = []
#num_batch = len(dfs) // batch + 1
start = int(0)
end = int(80)
st = int(0)
e = int(80)
null = None

start_t = timer()
results = Parallel(n_jobs=6, verbose=1, 
                   backend="multiprocessing")(delayed(get_shp)(n) for n in range(num_batch))
end_t = timer()
print(end_t - start_t)

retrieving:  80  /  414172


[Parallel(n_jobs=6)]: Using backend MultiprocessingBackend with 6 concurrent workers.


done
done
done
done
done
retrieving:  880  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  1680  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  2480  /  414172
done
done
done
done
done
done
SyntaxError
done
done
done
done
retrieving:  3280  /  414172
SyntaxError
done
SyntaxError
done
SyntaxError
done


[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:   28.3s


done
SyntaxError
done
done
done
SyntaxError
done
done
done
retrieving:  4080  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  4880  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  5680  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  6480  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  7280  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  8080  /  414172
done
done
done
done
done
done
done
done
done
done
done
retrieving:  8880  /  414172
done
done
done
done
done
done
done
done
done
retrieving:  9680  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  10480  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  11280  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  12080  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  12880  /  414172
done
done
done
done
done
done
done
done

...

done
done
done
done
done
done
done
retrieving:  64080  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  64880  /  414172
done
done
done
done
done
done
done
done
done
retrieving:  65680  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  66480  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  67280  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  68080  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  68880  /  414172
done
done
done
done
done
done
done
done
done
done
done
done
done
done
retrieving:  69680  /  414172
done
done
done
done
done
done
retrieving:  70480  /  414172
done
done
done
done
done
done
done
done
done
done
done
retrieving:  71280  /  414172
done
done
done
done
done
done
done
done
done
done
done
retrieving:  72080  /  414172
done
done
done
done
done
done
done
done
done
retrieving:  72880  /  414172
done
done
done
done
done
done
done
done
done
retrieving

[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed: 15.7min


done
done
done
done
done
done
done
done
done
retrieving:  100080  /  414172
done
done
done
done
done
done
done
done
done
done
done
retrieving:  100880  /  414172
done
done
done
done
done
done
done
done
done
retrieving:  101680  /  414172
done
done
done
done
done
done
done
done
retrieving:  102480  /  414172
done
done
done
done
done
done
done
done
done
done
done
done
retrieving:  103280  /  414172
done
done
done
done
done
done
done
done
done
done
done
retrieving:  104080  /  414172
done
done
done
done
done
done
done
done
retrieving:  104880  /  414172
done
done
done
done
done
done
done
done
done
done
done
retrieving:  105680  /  414172
done
done
done
done
done
done
done
done
retrieving:  106480  /  414172
done
done
done
done
done
done
done
done
done
done
done
retrieving:  107280  /  414172
done
done
done
done
done
done
done
done
done
done
done
retrieving:  108080  /  414172
done
done
done
done
done
done
done
done
done
retrieving:  108880  /  414172
done
done
done
done
done
done
done
don

[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed: 22.5min


done
done
done
done
done
done
retrieving:  144080  /  414172
done
done
done
done
done
done
done
done
done
done
done
retrieving:  144880  /  414172
done
done
done
done
done
done
done
done
done
done
done
done
retrieving:  145680  /  414172
done
done
done
done
done
done
done
retrieving:  146480  /  414172
done
done
done
done
done
done
done
done
done
done
done
done
retrieving:  147280  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  148080  /  414172
done
done
done
done
done
done
done
done
done
retrieving:  148880  /  414172
done
done
done
done
done
done
done
done
retrieving:  149680  /  414172
done
done
done
done
done
done
done
done
done
done
done
retrieving:  150480  /  414172
done
done
done
done
done
done
done
done
done
done
done
retrieving:  151280  /  414172
done
done
done
done
done
done
done
done
done
retrieving:  152080  /  414172
done
done
done
done
done
done
done
done
done
done
retrieving:  152880  /  414172
done
done
done
done
done
done
done
done
done
don

In [89]:
json_results = []
for r in results:
    json_results.append(eval(r))

In [95]:
json_results = json.dumps(json_results)

In [90]:
len(json_results)

13

In [88]:
eval(results[0])

[{'type': 'MultiPolygon',
  'coordinates': [[[[121.53293930498, 25.10798537579],
     [121.53293813819, 25.107973484727],
     [121.53291902686, 25.107975149112],
     [121.53291955135, 25.107985881972],
     [121.53292120427, 25.107999657121],
     [121.53290431963, 25.108000443119],
     [121.53290315158, 25.107988261931],
     [121.53282415276, 25.107993913169],
     [121.53281781448, 25.107928366966],
     [121.53298441398, 25.107916743413],
     [121.53299074983, 25.10798170936],
     [121.53293930498, 25.10798537579]]]],
  '縣市': '臺北市',
  '鄉鎮': '士林區',
  '段名': '芝山段一小段',
  '地號': 330000,
  'ymax': 25.108000443119,
  'ymin': 25.107916743413,
  'xmax': 121.53299074983,
  'xmin': 121.53281781448,
  'xcenter': 121.53290428216,
  'ycenter': 25.107958593266,
  'area_id': 'AE',
  'section_id': '0828',
  'land_id': 330000,
  'query': '臺北市,芝山段一小段,33'},
 {'type': 'MultiPolygon',
  'coordinates': [[[[121.5466256805, 25.025002359693],
     [121.54662299557, 25.024971632879],
     [121.5466614400

In [100]:
j = json.loads(json_results)
df_j = pd.io.json.json_normalize(j[0])

In [103]:
len(df_j) * 13

949

In [ ]:
len(results)